# iteration1

Data:
- interaktiot kaikkien muiden ryhmistä paitsi Alman kehittäjien
- interaktiot ryhmä, ei käyttäjäkohtaisia
- kaikki yritykset - konsernien tiedot pudotetaan pois
- metadatana yrityksistä maakuntakoodi ja toimialakoodi

Kysymyksiä:

1. Tuottaako malli mitenkään järkeviä suosituksia?
2. Warp vai bpr?
4. Ryhmäkoko rajoittamaton, min=10 vai min=50? (Tai siis miten vaikuttaa - käytännössähän pitäisi päättää millainen on minimiryhmäkoko, jolle suosituksia halutaan luoda)
3. Mitä tapahtuu, jos item_featuret jättää kokonaan pois?


## Importit

In [1]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k
from lightfm.evaluation import reciprocal_rank
from lightfm.data import Dataset

import numpy as np
import pandas as pd

import statistics

## Valitut metadatat yrityksille

In [2]:
SELECTED_COMPANY_FEATURES = ['location_region_code', 'industry_code']

## Ladataan yritysdata

In [4]:
COMPANIES_DF = pd \
        .read_csv('../data/prod_data_companies_more_data_2021_09_16.csv',
                  delimiter='\t',
                  na_values='(null)',
                  dtype={
                      'business_id': str,
                      'business_id_start_date': str,
                      'company_name': str,
                      'email_address': str,
                      'industry_code': str,
                      'language': str,
                      'company_form': str,
                      'company_form_code': str,
                      'domicile_code': str,
                      'region_code': str,
                      'location_municipality_code': str,
                      'location_region_code': str,
                      'company_status_code': str,
                      'balance_industry_code': str,
                      'personnel_class': str,
                      'turnover_class': str,
                      'statement_date': str,
                      'statement_length': float,
                      'target_year': str,
                      'turnover': float,
                      'raw_materials_use': float,
                      'current_ratio': float,
                      'personnel_expenses': float,
                      'investment_expenses_cash_flow': float,
                      'ebitda': float,
                      'operating_profit_per_turnover': float,
                      'ebit': float,
                      'turnover_growth_percent': float,
                      'ebit_per_turnover': float,
                      'short_term_sales_receivable': float,
                      'net_profit': float,
                      'net_profit_per_turnover': float,
                      'aggregate_equity': float,
                      'return_on_equity_percent': float,
                      'equity_ratio': float,
                      'long_term_sales_receivable': float,
                      'quick_ratio': float,
                      'buildings': float,
                      'return_on_investment_percent': float,
                      'total_assets': float,
                      'external_services': float,
                      'aggregate_current_assets': float,
                      'personnel_average': float,
                      'concern': str,
                      'bulk': str,
                      'performer_ranking_points': float,
                      'risk_rating_class': str,
                      'latest': str
                  }
                  )


## Poistetaan konsernit (duplikaattien poistamiseksi) ja valitaan halutut sarakkeet

In [6]:
COMPANIES_DF = COMPANIES_DF[COMPANIES_DF['concern'] != 'true']
COMPANIES_DF = COMPANIES_DF[['business_id', 'company_name'] + SELECTED_COMPANY_FEATURES]
COMPANIES_DF

,business_id,company_name,location_region_code,industry_code
0,31431209,Tmi Onerva Miettunen,01,43341
1,32087307,Koti Vaihtoon LKV Oy,01,68310
2,18601103,Teollisuuden Monialapalvelu T:mi Oksanen,NaN,68209
3,20469041,POLYBEAT,NaN,74909
4,24844507,SAHOJEN PUU,NaN,NaN
...,...,...,...,...
1337863,27527742,Ingenium Partners Oy,NaN,70220
1337864,31204145,Rainmaker Voima Oy,01,78300
1337865,30137091,We Build Parks Finland Oy,19,42999
1337866,32150097,Orankien Osuuskunta,01,62010


## Prefiksoidaan metadatakoodit ja käsitellään puuttuvat kentät

In [7]:
def transform_str_to_keyword(val, col_name):
    if (pd.isnull(val)):
        return col_name + '+' + 'NaN'
    return col_name + '+' + val

for col_name in SELECTED_COMPANY_FEATURES:
    COMPANIES_DF[col_name] = COMPANIES_DF[col_name].apply(lambda x: transform_str_to_keyword(x, col_name))

COMPANIES_DF

,business_id,company_name,location_region_code,industry_code
0,31431209,Tmi Onerva Miettunen,location_region_code+01,industry_code+43341
1,32087307,Koti Vaihtoon LKV Oy,location_region_code+01,industry_code+68310
2,18601103,Teollisuuden Monialapalvelu T:mi Oksanen,location_region_code+NaN,industry_code+68209
3,20469041,POLYBEAT,location_region_code+NaN,industry_code+74909
4,24844507,SAHOJEN PUU,location_region_code+NaN,industry_code+NaN
...,...,...,...,...
1337863,27527742,Ingenium Partners Oy,location_region_code+NaN,industry_code+70220
1337864,31204145,Rainmaker Voima Oy,location_region_code+01,industry_code+78300
1337865,30137091,We Build Parks Finland Oy,location_region_code+19,industry_code+42999
1337866,32150097,Orankien Osuuskunta,location_region_code+01,industry_code+62010


## Käsitellään yritysdataa LightFM:n Dataset-olion luontia varten

In [8]:
ITEM_IDS = list(COMPANIES_DF['business_id'].unique())

item_features_tmp = [COMPANIES_DF[feature].unique() for feature in SELECTED_COMPANY_FEATURES]

ITEM_FEATURE_LABELS = [item for sublist in item_features_tmp for item in sublist]

ITEM_FEATURES = [(company['business_id'], 
                  [company[feature] for feature in SELECTED_COMPANY_FEATURES])
                     for company in COMPANIES_DF.to_dict(orient='records')]

print(ITEM_FEATURES[0])
print(len(ITEM_FEATURES))
print(len(ITEM_FEATURE_LABELS))

('31431209', ['location_region_code+01', 'industry_code+43341'])
1334601
838


## Ladataan vuorovaikutusdata

In [9]:
interactions_tmp = pd \
    .read_csv('../data/interactions_2021_08_19.csv',
             delimiter='\t',
             dtype={
                 'group_id': 'string',
                 'business_id': 'string',
                 'owner': 'string'
             })

# Poistetaan vuorovaikutusdatasta sellaiset y-tunnukset, joita ei löydy kohteista
INTERACTIONS_DF = interactions_tmp[interactions_tmp.business_id.isin(ITEM_IDS)]

## Luodaan vuorovaikutusdatasta versiot erilaisilla minimiryhmäko'oilla

In [10]:
group_sizes = INTERACTIONS_DF['group_id'].value_counts()
group_sizes_df = pd.DataFrame({'group_id': group_sizes.index, 'group_size': group_sizes.values})

INTERACTIONS_WITH_GROUP_SIZES_DF = INTERACTIONS_DF.merge(group_sizes_df, on='group_id')


##### Minimiryhmäkoko = 10 #####

INTERACTIONS_10_DF = INTERACTIONS_WITH_GROUP_SIZES_DF[INTERACTIONS_WITH_GROUP_SIZES_DF.group_size >= 10]


##### Minimiryhmäkoko = 50

INTERACTIONS_50_DF = INTERACTIONS_WITH_GROUP_SIZES_DF[INTERACTIONS_WITH_GROUP_SIZES_DF.group_size >= 50]


In [19]:
def print_interactions_meta_data(interactions_df):
    print('ryhmiä: {groups}, interaktioita {interactions}, yrityksiä {companies}'
          .format(groups=len(list(interactions_df['group_id'].unique())),
                  interactions=interactions_df.shape[0], 
                  companies=len(list(interactions_df['business_id'].unique()))))

print('----- group_size>=1 -----')
print_interactions_meta_data(INTERACTIONS_DF)

print('\n----- group_size>=10 -----')
print_interactions_meta_data(INTERACTIONS_10_DF)

print('\n----- group_size>=50 -----')
print_interactions_meta_data(INTERACTIONS_50_DF)

----- group_size>=1 -----
ryhmiä: 1399, interaktioita 525513, yrityksiä 140677

----- group_size>=10 -----
ryhmiä: 1015, interaktioita 524053, yrityksiä 140604

----- group_size>=50 -----
ryhmiä: 722, interaktioita 517150, yrityksiä 140284


## Luodaan LightFM:n ymmärtämät Dataset-oliot eri ryhmäko'oille

In [12]:
def create_dataset(interactions_df):
    dataset = Dataset(user_identity_features=False, item_identity_features=False)
    
    interactions = [(interaction['group_id'], interaction['business_id']) 
                for interaction in interactions_df.to_dict(orient='records')]
    
    user_ids = list(set(interactions_df['group_id'].values))

    dataset.fit(users=user_ids, items=ITEM_IDS, item_features=ITEM_FEATURE_LABELS)
    
    (interactions_ds, weights_ds) = dataset.build_interactions(interactions)
    
    item_features_ds = dataset.build_item_features(ITEM_FEATURES, normalize=False)

    # USER_MAP_DS = dataset.mapping()[0]
    # ITEM_MAP_DS = dataset.mapping()[2]
    # ITEM_FEATURE_MAP_DS = dataset.mapping()[3]
    
    return (interactions_ds, item_features_ds)    

In [13]:
(INTERACTIONS_10_DS, ITEM_FEATURES_10_DS) = create_dataset(INTERACTIONS_10_DF)

In [20]:
INTERACTIONS_10_DS

<1015x1334601 sparse matrix of type '<class 'numpy.int32'>'
	with 524053 stored elements in COOrdinate format>

In [21]:
ITEM_FEATURES_10_DS

<1334601x838 sparse matrix of type '<class 'numpy.float32'>'
	with 2669202 stored elements in Compressed Sparse Row format>